In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
import random
from sklearn.model_selection import train_test_split
from torchvision.io import read_image
from PIL import Image
from tqdm import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt

In [48]:
class ToRGB(object):
    def __call__(self, img):
        img = img.convert('RGB')
        return img

In [49]:
transform_train = transforms.Compose([
    ToRGB(),  # Применяем наш класс для преобразования изображений
    transforms.Resize((224, 224)),  # Изменение размера изображений
    transforms.RandomHorizontalFlip(),  # Случайное горизонтальное отражение
    transforms.RandomRotation(15),  # Случайная ротация
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Случайные изменения яркости, контраста и насыщенности
    transforms.ToTensor(),  # Преобразование изображений в тензоры
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

transform_val_test = transforms.Compose([
    ToRGB(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [50]:
root_dir = r'C:\Users\home\Desktop\train_minprirodi_Parnokopitnie'
categories = ['kosulya', 'kabarga', 'olen']

In [51]:
# Пустые множества для хранения уникальных путей и меток
all_images = set()
all_labels = []

# Функция для проверки изображения
def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()  # Проверка целостности изображения
        return True
    except (IOError, OSError):
        return False

In [52]:
for label, category in enumerate(categories):
    category_dir = os.path.join(root_dir, category)
    for file_name in os.listdir(category_dir):
        file_path = os.path.join(category_dir, file_name)
        if file_path.endswith(('jpg', 'jpeg', 'png')) and is_valid_image(file_path):
            all_images.add(file_path)
            all_labels.append(label)


In [53]:
all_images = list(all_images)

# Разделение данных на тренировочный, валидационный и тестовый наборы
train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.2, stratify=all_labels, random_state=42)
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.25, stratify=train_labels, random_state=42)

In [54]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image_path = self.images[idx]
        try:
            image = Image.open(image_path)
            image.verify()  # Проверка целостности изображения
            image = Image.open(image_path)  # Переоткрытие изображения после проверки
        except (IOError, OSError):
            print(f"Warning: Cannot identify image file {image_path}. Skipping.")
            return None, None
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Создание датасетов с трансформациями
train_dataset = CustomDataset(train_images, train_labels, transform=transform_train)
val_dataset = CustomDataset(val_images, val_labels, transform=transform_val_test)
test_dataset = CustomDataset(test_images, test_labels, transform=transform_val_test)

# Функция для удаления None из батча данных
def collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [55]:
# Загрузка предобученной модели ResNet18
model = models.resnet50(pretrained=True)

# Замена последнего слоя на новый с правильным числом выходов
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(categories))

# Перенос модели на устройство (CPU или GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


C:\Users\home\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\home\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\home/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:20<00:00, 4.88MB/s]


In [ ]:
num_epochs = 20  # Установите нужное количество эпох

for epoch in range(num_epochs):
    model.train()  # Переводим модель в режим обучения
    running_loss = 0.0  # Обнуляем текущее значение функции потерь
    
    # Проходим по тренировочному набору данных
    for inputs, labels in (pbar := tqdm(train_loader)):
        inputs, labels = inputs.to(device), labels.to(device)  # Передаем данные на устройство для обучения
        
        optimizer.zero_grad()  # Обнуляем градиенты параметров
        
        outputs = model(inputs)  # Прямой проход через модель
        
        loss = criterion(outputs, labels)  # Вычисляем значение функции потерь
        loss.backward()  # Обратное распространение ошибки
        optimizer.step()  # Обновление весов модели
        
        running_loss += loss.item() * inputs.size(0)  # Обновляем значение функции потерь
    
    # Выводим среднее значение функции потерь для эпохи
    train_loss = running_loss / len(train_loader.dataset)
    
    # Проверяем модель на валидационном наборе данных
    model.eval()  # Переводим модель в режим оценки
    val_loss = 0.0
    correct = 0
    total = 0
    
    # Отключаем вычисление градиентов
    with torch.no_grad():
        for inputs, labels in (pbar := tqdm(val_loader)):
            inputs, labels = inputs.to(device), labels.to(device)  # Передаем данные на устройство для обучения
            
            outputs = model(inputs)  # Прямой проход через модель
            
            loss = criterion(outputs, labels)  # Вычисляем значение функции потерь
            val_loss += loss.item() * inputs.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = 100 * correct / total
    pbar.set_description(f"loss: {val_loss:.4f}\taccuracy: {correct:.3f}")
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Training Loss: {train_loss:.4f}, "
          f"Validation Loss: {val_loss:.4f}, "
          f"Validation Accuracy: {val_accuracy:.2f}%")


 30%|██▉       | 41/138 [06:35<15:35,  9.64s/it]

In [ ]:
# Оценка модели на тестовом наборе данных с выводом изображений и предсказаний
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Выводим изображения с предсказаниями
        for i in range(inputs.size(0)):
            img = inputs[i].cpu().permute(1, 2, 0).numpy()
            img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]  # Де-нормализация
            img = img.clip(0, 1)

            plt.imshow(img)
            plt.title(f"Predicted: {categories[predicted[i]]}, Actual: {categories[labels[i]]}")
            plt.show()
            print(f"Predicted: {categories[predicted[i]]}, Actual: {categories[labels[i]]}")

test_loss = test_loss / len(test_loader.dataset)
test_accuracy = 100 * correct / total

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")